In [57]:
import pandas as pd

# Import data

In [58]:
df_electricity=pd.read_csv("./raw_data/electricity_prices.csv")
df_gas=pd.read_csv("./raw_data/gas_prices.csv")
# Import train data
df_train = pd.read_csv("./raw_data/train.csv")

# Changing data format

### Manipulating dates
Check the number of unique values for the date column in the electricity
By checking only the unique dates, it will reduce the number of calculation when we will compare the date between the electricity df and the train df

In [59]:
print("Number of unique values in elec :", df_electricity["origin_date"].nunique())
print("Number of unique values in gas :", df_gas["origin_date"].nunique())

Number of unique values in elec : 15286
Number of unique values in gas : 637


# Feature Engineering

## Adding gas and electricity prices in df_train
First of all we can see that the dataframe for the gas, do not use the same date format than the electricity and the train dataframes date format.
So we will be able to merge the info from the electricity dataframe, based on the datetime from the train dataframe.
But for the gas dataframe, we will need to add a formated column in train df. A column with the same format as the date in the gas df. We keep the date, we just remove the time.
After this manipulation, we can merge the gas prices too.

### Add elec prices

In [60]:
## Merge elec prices
# Creating a new column based on datetime
df_train["forecast_date"] = df_train["datetime"]
# Selecting the infos we need to merge in the electricity df
df_elec_prices = df_electricity.filter(['euros_per_mwh', 'forecast_date'], axis=1)
# Merging
df_train = df_train.merge(df_elec_prices, how="left", on="forecast_date", suffixes=('', '_elec'))
df_train.rename(columns={"euros_per_mwh": "euros_per_mwh_elec"}, inplace=True)


In [61]:
df_train.isna().sum()

county                   0
is_business              0
product_type             0
target                 528
is_consumption           0
datetime                 0
data_block_id            0
row_id                   0
prediction_unit_id       0
forecast_date            0
euros_per_mwh_elec    3386
dtype: int64

### Add gas prices

In [62]:
df_train["simplified_date"] = pd.to_datetime(df_train["forecast_date"]).dt.strftime('%Y-%m-%d')
df_train["simplified_date"]

0          2021-09-01
1          2021-09-01
2          2021-09-01
3          2021-09-01
4          2021-09-01
              ...    
2018347    2023-05-31
2018348    2023-05-31
2018349    2023-05-31
2018350    2023-05-31
2018351    2023-05-31
Name: simplified_date, Length: 2018352, dtype: object

In [63]:
# Selecting the infos we need to merge in the gas df
df_gas_prices = df_gas.filter(['lowest_price_per_mwh', 'highest_price_per_mwh', 'forecast_date'])
df_gas_prices.rename(columns={"forecast_date": "simplified_date"}, inplace=True)

# Merging
df_train = df_train.merge(df_gas_prices, how="left", on="simplified_date")
df_train.rename(columns={"lowest_price_per_mwh": "lowest_price_per_mwh_gas", "highest_price_per_mwh": "highest_price_per_mwh_gas"}, inplace=True)

df_train

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id,forecast_date,euros_per_mwh_elec,simplified_date,lowest_price_per_mwh_gas,highest_price_per_mwh_gas
0,0,0,1,0.713,0,2021-09-01 00:00:00,0,0,0,2021-09-01 00:00:00,92.51,2021-09-01,45.23,46.32
1,0,0,1,96.590,1,2021-09-01 00:00:00,0,1,0,2021-09-01 00:00:00,92.51,2021-09-01,45.23,46.32
2,0,0,2,0.000,0,2021-09-01 00:00:00,0,2,1,2021-09-01 00:00:00,92.51,2021-09-01,45.23,46.32
3,0,0,2,17.314,1,2021-09-01 00:00:00,0,3,1,2021-09-01 00:00:00,92.51,2021-09-01,45.23,46.32
4,0,0,3,2.904,0,2021-09-01 00:00:00,0,4,2,2021-09-01 00:00:00,92.51,2021-09-01,45.23,46.32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018347,15,1,0,197.233,1,2023-05-31 23:00:00,637,2018347,64,2023-05-31 23:00:00,NaN,2023-05-31,NaN,NaN
2018348,15,1,1,0.000,0,2023-05-31 23:00:00,637,2018348,59,2023-05-31 23:00:00,NaN,2023-05-31,NaN,NaN
2018349,15,1,1,28.404,1,2023-05-31 23:00:00,637,2018349,59,2023-05-31 23:00:00,NaN,2023-05-31,NaN,NaN
2018350,15,1,3,0.000,0,2023-05-31 23:00:00,637,2018350,60,2023-05-31 23:00:00,NaN,2023-05-31,NaN,NaN


In [64]:
df_train.isna().sum()

county                          0
is_business                     0
product_type                    0
target                        528
is_consumption                  0
datetime                        0
data_block_id                   0
row_id                          0
prediction_unit_id              0
forecast_date                   0
euros_per_mwh_elec           3386
simplified_date                 0
lowest_price_per_mwh_gas     3120
highest_price_per_mwh_gas    3120
dtype: int64

### Add clients

In [65]:
# Import data
df_client = pd.read_csv("./raw_data/client.csv")
# Create a sub DataFrame
df_client_filtered = df_client.filter(['is_business', 'product_type', 'date', 'installed_capacity', 'county', 'eic_count'])
# Rename "date" column for the merge
df_client_filtered.rename(columns={"date": "simplified_date"}, inplace=True)
# Group by mean the date and the product_type
df_client_grouped = df_client_filtered.groupby(["simplified_date", "product_type"]).mean()

# Merge with the df_train
df_train = df_train.merge(df_client_grouped, how="left", on=["simplified_date", "product_type"], suffixes=('', '_clt'))
df_train

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id,forecast_date,euros_per_mwh_elec,simplified_date,lowest_price_per_mwh_gas,highest_price_per_mwh_gas,is_business_clt,installed_capacity,county_clt,eic_count
0,0,0,1,0.713,0,2021-09-01 00:00:00,0,0,0,2021-09-01 00:00:00,92.51,2021-09-01,45.23,46.32,0.409091,360.281364,7.909091,20.863636
1,0,0,1,96.590,1,2021-09-01 00:00:00,0,1,0,2021-09-01 00:00:00,92.51,2021-09-01,45.23,46.32,0.409091,360.281364,7.909091,20.863636
2,0,0,2,0.000,0,2021-09-01 00:00:00,0,2,1,2021-09-01 00:00:00,92.51,2021-09-01,45.23,46.32,0.400000,110.180000,7.800000,8.600000
3,0,0,2,17.314,1,2021-09-01 00:00:00,0,3,1,2021-09-01 00:00:00,92.51,2021-09-01,45.23,46.32,0.400000,110.180000,7.800000,8.600000
4,0,0,3,2.904,0,2021-09-01 00:00:00,0,4,2,2021-09-01 00:00:00,92.51,2021-09-01,45.23,46.32,0.533333,1924.952333,7.400000,87.433333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018347,15,1,0,197.233,1,2023-05-31 23:00:00,637,2018347,64,2023-05-31 23:00:00,NaN,2023-05-31,NaN,NaN,NaN,NaN,NaN,NaN
2018348,15,1,1,0.000,0,2023-05-31 23:00:00,637,2018348,59,2023-05-31 23:00:00,NaN,2023-05-31,NaN,NaN,NaN,NaN,NaN,NaN
2018349,15,1,1,28.404,1,2023-05-31 23:00:00,637,2018349,59,2023-05-31 23:00:00,NaN,2023-05-31,NaN,NaN,NaN,NaN,NaN,NaN
2018350,15,1,3,0.000,0,2023-05-31 23:00:00,637,2018350,60,2023-05-31 23:00:00,NaN,2023-05-31,NaN,NaN,NaN,NaN,NaN,NaN


In [66]:
df_train.isna().sum()

county                          0
is_business                     0
product_type                    0
target                        528
is_consumption                  0
datetime                        0
data_block_id                   0
row_id                          0
prediction_unit_id              0
forecast_date                   0
euros_per_mwh_elec           3386
simplified_date                 0
lowest_price_per_mwh_gas     3120
highest_price_per_mwh_gas    3120
is_business_clt              6240
installed_capacity           6240
county_clt                   6240
eic_count                    6240
dtype: int64

### Add weather to final df

In [67]:
# Historical weather
df_historical_weather = pd.read_csv("./raw_data/historical_weather.csv")
df_historical_weather_grouped = df_historical_weather.groupby(["datetime"]).mean()

df_historical_weather_grouped.drop(columns=["longitude", "latitude", "data_block_id"], inplace=True)

df_train = df_train.merge(df_historical_weather_grouped, how="left", on="datetime")

# Forecast weather
df_forecast_weather = pd.read_csv("./raw_data/forecast_weather.csv")
df_forecast_weather.drop(columns=["origin_datetime", "longitude", "latitude", "data_block_id"], inplace=True)
df_forecast_weather.rename(columns={"forecast_datetime": "datetime"}, inplace=True)
df_forecast_weather_grouped = df_forecast_weather.groupby(["datetime"]).mean()
df_train = df_train.merge(df_forecast_weather_grouped, how="left", on="datetime", suffixes=('', '_fc'))
df_train

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id,forecast_date,...,cloudcover_high_fc,cloudcover_low_fc,cloudcover_mid_fc,cloudcover_total_fc,10_metre_u_wind_component,10_metre_v_wind_component,direct_solar_radiation_fc,surface_solar_radiation_downwards,snowfall_fc,total_precipitation
0,0,0,1,0.713,0,2021-09-01 00:00:00,0,0,0,2021-09-01 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,0,1,96.590,1,2021-09-01 00:00:00,0,1,0,2021-09-01 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0,2,0.000,0,2021-09-01 00:00:00,0,2,1,2021-09-01 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,0,2,17.314,1,2021-09-01 00:00:00,0,3,1,2021-09-01 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,0,3,2.904,0,2021-09-01 00:00:00,0,4,2,2021-09-01 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018347,15,1,0,197.233,1,2023-05-31 23:00:00,637,2018347,64,2023-05-31 23:00:00,...,0.50497,0.016986,0.031082,0.526855,4.432824,-2.40581,0.010794,0.0,0.0,0.0
2018348,15,1,1,0.000,0,2023-05-31 23:00:00,637,2018348,59,2023-05-31 23:00:00,...,0.50497,0.016986,0.031082,0.526855,4.432824,-2.40581,0.010794,0.0,0.0,0.0
2018349,15,1,1,28.404,1,2023-05-31 23:00:00,637,2018349,59,2023-05-31 23:00:00,...,0.50497,0.016986,0.031082,0.526855,4.432824,-2.40581,0.010794,0.0,0.0,0.0
2018350,15,1,3,0.000,0,2023-05-31 23:00:00,637,2018350,60,2023-05-31 23:00:00,...,0.50497,0.016986,0.031082,0.526855,4.432824,-2.40581,0.010794,0.0,0.0,0.0


### Split datetime in multiple columns
Split into 4 columns the datetime (year, month, day, hour)

In [68]:
import numpy as np

df_train["hour"] = pd.to_datetime(df_train["datetime"]).dt.hour
df_train['sin_hour'] = np.sin(2 * np.pi * (df_train['hour'] / max(df_train['hour'])))
df_train['cos_hour'] = np.cos(2 * np.pi * (df_train['hour'] / max(df_train['hour'])))

df_train["day"] = pd.to_datetime(df_train["datetime"]).dt.day
df_train['sin_day'] = np.sin(2 * np.pi * (df_train['day'] / max(df_train['day'])))
df_train['cos_day'] = np.cos(2 * np.pi * (df_train['day'] / max(df_train['day'])))

df_train["month"] = pd.to_datetime(df_train["datetime"]).dt.month
df_train['sin_month'] = np.sin(2 * np.pi * (df_train['month'] / max(df_train['month'])))
df_train['cos_month'] = np.cos(2 * np.pi * (df_train['month'] / max(df_train['month'])))

df_train["year"] = pd.to_datetime(df_train["datetime"]).dt.year

df_train.drop(columns=["datetime", "simplified_date", "row_id", "forecast_date", "month", "day", "hour"], inplace=True)

In [69]:
df_train.to_csv("./formated_data/train.csv",index=False)